In [1]:
import os
import gc
import cv2
import timm
import random
import numpy as np
import pandas as pd
from glob import glob

import torch
import torch.nn as nn
from torch import Tensor
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.autonotebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import wandb

import warnings
warnings.filterwarnings(action='ignore') 

/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [3]:
CFG = {
    # 원본 (512, 384)
    'IMG_SIZE_H': 192,
    'IMG_SIZE_W': 128,
    'EPOCHS': 100,
    'LEARNING_RATE': 1e-5,
    'BATCH_SIZE': 32,
    'SEED': 909,
    'PAATIENCE_LIMIT': 5,
    'MODEL': 'resnet50',
    'LOSS': 'FocalLoss',
}

In [4]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="mask-classification",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": CFG['LEARNING_RATE'],
    "architecture": CFG['MODEL'],
    "epochs": CFG['EPOCHS'],
    "batch_size" : CFG['BATCH_SIZE'],
    "loss" : CFG['LOSS'],
    },
    
    name=f"{CFG['MODEL']}_{CFG['LOSS']}_{CFG['IMG_SIZE_H']}-{CFG['IMG_SIZE_W']}"
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kgw5430. Use `wandb login --relogin` to force relogin


In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

### Focal Loss

In [6]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, weight=self.weight, reduction=self.reduction)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1-pt)**self.gamma * ce_loss).mean()
        return focal_loss

### Custom Dataset

In [7]:
# class CustomDataset(Dataset):
#     def __init__(self, img_path_list, label_list, transforms=None):
#         self.img_path_list = img_path_list
#         self.label_list = label_list
#         self.transforms = transforms
        
#     def __getitem__(self, index):
#         img_path = self.img_path_list[index]
#         image = cv2.imread(img_path)
        
#         if self.transforms is not None:
#             image = self.transforms(image=image)['image']
        
#         if self.label_list is not None:
#             label = self.label_list[index]
#             return image, label
#         else:
#             return image
        
#     def __len__(self):
#         return len(self.img_path_list)

In [7]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        self.feature = []
        
        for img_path in self.img_path_list:
            image = cv2.imread(img_path)
            if self.transforms is not None:
                image = self.transforms(image=image)['image']
            self.feature.append(image)
        
    def __getitem__(self, index):        
        if self.label_list is not None:
            return self.feature[index], self.label_list[index]
        else:
            return self.feature[index]
        
    def __len__(self):
        return len(self.img_path_list)

In [8]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE_W'],CFG['IMG_SIZE_W']),
                            A.HorizontalFlip(always_apply=False, p=0.5),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE_W'],CFG['IMG_SIZE_W']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

### Model Define

In [10]:
class Mask(nn.Module):
    def __init__(self, num_classes=3, pretrained=True):
        super(Mask, self).__init__()
        self.num_classes = num_classes
        self.pretrained = pretrained
        
        # self.model = timm.create_model('efficientnet_b0', pretrained=self.pretrained)
        self.model = timm.create_model('resnet50', pretrained=self.pretrained)
        self.fc = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(1000, 512),
                               nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(512, num_classes),
                               )

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        x = F.sigmoid(x)
        return x

In [11]:
class Gender(nn.Module):
    def __init__(self, num_classes=2, pretrained=True):
        super(Gender, self).__init__()
        self.num_classes = num_classes
        self.pretrained = pretrained
        
        # self.model = timm.create_model('efficientnet_b0', pretrained=self.pretrained)
        self.model = timm.create_model('resnet50', pretrained=self.pretrained)
        self.fc = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(1000, 512),
                               nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(512, num_classes),
                               )

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        x = F.sigmoid(x)
        return x

In [12]:
class Age(nn.Module):
    def __init__(self, num_classes=3, pretrained=True):
        super(Age, self).__init__()
        self.num_classes = num_classes
        self.pretrained = pretrained
        
        # self.model = timm.create_model('efficientnet_b0', pretrained=self.pretrained)
        self.model = timm.create_model('resnet50', pretrained=self.pretrained)
        self.fc = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(1000, 512),
                               nn.Dropout(p=0.2, inplace=True),
                               nn.Linear(512, num_classes),
                               )

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        x = F.sigmoid(x)
        return x

### Train

In [13]:
def train(model, optimizer, train_loader, val_loader, scheduler, device, label_weights, model_name):
    model.to(device)
    # criterion = nn.CrossEntropyLoss(weight=label_weights).to(device)
    criterion = FocalLoss(weight=label_weights).to(device)
    
    best_val_loss = float('inf')
    best_val_score = 0
    best_model = None
    
    # Early Stop
    patience_limit = CFG['PAATIENCE_LIMIT']
    patience = 0
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
             
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        wandb.log({f"Epoch": epoch, f"{model_name}/Train_Loss": _train_loss, f"{model_name}/Val_Loss": _val_loss, f"{model_name}/Val_ACC": _val_acc})
        
        if scheduler is not None:
            scheduler.step(_val_loss)
            
        if best_val_loss > _val_loss:
            best_val_loss = _val_loss
            best_val_score = _val_acc
            best_model = model
            patience = 0
        else:
            patience += 1
            if patience >= patience_limit:
                break

    print(f'Best Loss : [{best_val_loss:.5f}] Best ACC : [{best_val_score:.5f}]')
    return best_model, best_val_score, best_val_loss

In [14]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            logit = model(imgs)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
        _val_acc = f1_score(trues, preds, average='weighted')
    
    return _val_loss, _val_acc

### Load Data

In [15]:
train_dir = '/opt/ml/input/data/train'

In [16]:
df = pd.read_csv(train_dir + '/train.csv')

# drop or edit
error_data = [2399, 2400, 1912, 764]
for error in error_data:
    df = df.drop(error)
    
df = df.reset_index(drop=True)

In [17]:
num2class = ['incorrect_mask', 'mask1', 'mask2', 'mask3', 'mask4', 'mask5', 'normal']
class2num = {k: v for v, k in enumerate(num2class)}

In [18]:
all_img_path = glob(os.path.join(train_dir, 'images', '*', '*'))
print(len(all_img_path))

18900


In [19]:
df_mask = pd.DataFrame(columns=['id', 'path', 'label'])
df_mask['path'] = all_img_path

df_mask['id'] = df_mask['path'].apply(lambda x : str(x).split('/')[7][:6])
df_mask['label'] = df_mask['path'].apply(lambda x : str(x).split('/')[-1])

df_mask['label'].loc[df_mask['label'].str.contains('incorrect')] = "2"
df_mask['label'].loc[df_mask['label'].str.contains('normal')] = "1"
df_mask['label'].loc[df_mask['label'].str.contains('mask')] = "0"
df_mask['label'] = df_mask['label'].astype('int')

print(df_mask['label'].value_counts())
df_mask['path'][0]

0    13500
2     2700
1     2700
Name: label, dtype: int64


'/opt/ml/input/data/train/images/001148_female_Asian_58/mask2.jpg'

In [20]:
df_gender = pd.DataFrame(columns=['id', 'path', 'label'])
df_gender['path'] = all_img_path
df_gender['id'] = df_gender['path'].apply(lambda x : str(x).split('/')[7][:6])
df_gender['label'] = df_gender['path'].apply(lambda x : (str(x).split('/')[7]).split('_')[1])

df_gender['label'].loc[df_gender['label'] == "female"] = 0
df_gender['label'].loc[df_gender['label'] == "male"] = 1

df_gender['label'].value_counts()

0    11606
1     7294
Name: label, dtype: int64

In [21]:
df_age = pd.DataFrame(columns=['id', 'path', 'label'])
df_age['path'] = all_img_path
df_age['id'] = df_age['path'].apply(lambda x : str(x).split('/')[7][:6])
df_age['label'] = df_age['path'].apply(lambda x : int((str(x).split('/')[7]).split('_')[3]))

df_age['label'].loc[df_age['label'] < 30] = 0
df_age['label'].loc[(df_age['label'] >= 30) & (df_age['label'] < 60)] = 1
df_age['label'].loc[df_age['label'] >= 60] = 2

df_age['label'].value_counts()

0    8967
1    8589
2    1344
Name: label, dtype: int64

### Class Weights

In [22]:
from sklearn.utils.class_weight import compute_class_weight

mask_labels = [i for i in df_mask['label']]
mask_labels.sort()
mask_weights = compute_class_weight(class_weight='balanced', classes=np.unique(df_mask['label']), y=mask_labels)
mask_weights = torch.FloatTensor(mask_weights).to(device)

gender_labels = [i for i in df_gender['label']]
gender_labels.sort()
gender_weights = compute_class_weight(class_weight='balanced', classes=np.unique(df_gender['label']), y=gender_labels)
gender_weights = torch.FloatTensor(gender_weights).to(device)

age_labels = [i for i in df_age['label']]
age_labels.sort()
age_weights = compute_class_weight(class_weight='balanced', classes=np.unique(df_age['label']), y=age_labels)
age_weights = torch.FloatTensor(age_weights).to(device)

mask_weights, gender_weights, age_weights

(tensor([0.4667, 2.3333, 2.3333], device='cuda:0'),
 tensor([0.8142, 1.2956], device='cuda:0'),
 tensor([0.7026, 0.7335, 4.6875], device='cuda:0'))

In [23]:
train_mask, val_mask, _, _ = train_test_split(df_mask, df_mask['label'], test_size=0.2, random_state=CFG['SEED'], stratify=df_mask['label'])
print(len(train_mask), len(val_mask))

train_gender, val_gender, _, _ = train_test_split(df_gender, df_gender['label'], test_size=0.2, random_state=CFG['SEED'], stratify=df_gender['label'])
print(len(train_gender), len(val_gender))

train_age, val_age, _, _ = train_test_split(df_age, df_age['label'], test_size=0.2, random_state=CFG['SEED'], stratify=df_age['label'])
print(len(train_age), len(val_age))

15120 3780
15120 3780
15120 3780


### Train

In [24]:
project_idx = len(glob('/opt/ml/models/*'))

In [25]:
model_name = 'Mask'

train_dataset = CustomDataset(train_mask['path'].values, train_mask['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=8)

val_dataset = CustomDataset(val_mask['path'].values, val_mask['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=8)

model = Mask()
model = nn.DataParallel(model)
model.eval()

optimizer = torch.optim.AdamW(params = model.parameters(), lr=CFG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs',min_lr=1e-9, verbose=True)
mask_model, mask_best_score, mask_best_loss = train(model, optimizer, train_loader, val_loader, scheduler, device, mask_weights, model_name)

os.makedirs(f'/opt/ml/models/{project_idx}/{model_name}', exist_ok=True)
torch.save(mask_model.module.state_dict(), f'/opt/ml/models/{project_idx}/{model_name}/[{CFG["MODEL"]}]_[score{mask_best_score:.4f}]_[loss{mask_best_loss:.4f}].pt')

gc.collect() # python 자원 관리 
torch.cuda.empty_cache() # gpu 자원관리


Epoch [1], Train Loss : [0.49204] Val Loss : [0.48759] Val ACC : [0.16864]



Epoch [2], Train Loss : [0.42085] Val Loss : [0.22408] Val ACC : [0.49044]



Epoch [3], Train Loss : [0.22066] Val Loss : [0.17783] Val ACC : [0.18299]



Epoch [4], Train Loss : [0.17926] Val Loss : [0.16314] Val ACC : [0.26247]



Epoch [5], Train Loss : [0.16714] Val Loss : [0.16045] Val ACC : [0.41856]


...


Epoch [13], Train Loss : [0.15588] Val Loss : [0.15560] Val ACC : [0.79519]



Epoch [14], Train Loss : [0.15572] Val Loss : [0.15547] Val ACC : [0.92527]



Epoch [15], Train Loss : [0.15571] Val Loss : [0.15530] Val ACC : [0.90962]



Epoch [16], Train Loss : [0.15548] Val Loss : [0.15506] Val ACC : [0.87813]


KeyboardInterrupt: 

In [ ]:
model_name = 'Gender'

train_dataset = CustomDataset(train_gender['path'].values, train_gender['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=8)

val_dataset = CustomDataset(val_gender['path'].values, val_gender['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=8)

model = Gender()
model = nn.DataParallel(model)
model.eval()

optimizer = torch.optim.AdamW(params = model.parameters(), lr=CFG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs',min_lr=1e-9, verbose=True)
gender_model, gender_best_score, gender_best_loss = train(model, optimizer, train_loader, val_loader, scheduler, device, gender_weights, model_name)

os.makedirs(f'/opt/ml/models/{project_idx}/{model_name}', exist_ok=True)
torch.save(gender_model.module.state_dict(), f'/opt/ml/models/{project_idx}/{model_name}/[{CFG["MODEL"]}]_[score{gender_best_score:.4f}]_[loss{gender_best_loss:.4f}].pt')

gc.collect() # python 자원 관리 
torch.cuda.empty_cache() # gpu 자원관리


Epoch [1], Train Loss : [0.17402] Val Loss : [0.17319] Val ACC : [0.46718]



Epoch [2], Train Loss : [0.17310] Val Loss : [0.17324] Val ACC : [0.71410]



Epoch [3], Train Loss : [0.15046] Val Loss : [0.08081] Val ACC : [0.83342]



Epoch [4], Train Loss : [0.07578] Val Loss : [0.05415] Val ACC : [0.89307]



Epoch [5], Train Loss : [0.05379] Val Loss : [0.04573] Val ACC : [0.91192]


...


Epoch [36], Train Loss : [0.02527] Val Loss : [0.02915] Val ACC : [0.97221]
Epoch    36: reducing learning rate of group 0 to 6.2500e-07.



Epoch [37], Train Loss : [0.02521] Val Loss : [0.02895] Val ACC : [0.97380]



Epoch [38], Train Loss : [0.02531] Val Loss : [0.02878] Val ACC : [0.97096]



Epoch [39], Train Loss : [0.02529] Val Loss : [0.02926] Val ACC : [0.97168]
Epoch    39: reducing learning rate of group 0 to 3.1250e-07.



Epoch [40], Train Loss : [0.02523] Val Loss : [0.02867] Val ACC : [0.97277]
Best Loss : [0.02845] Best ACC : [0.97306]


In [ ]:
model_name = 'Age'

train_dataset = CustomDataset(train_age['path'].values, train_age['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=8)

val_dataset = CustomDataset(val_age['path'].values, val_age['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=8)

model = Age()
model = nn.DataParallel(model)
model.eval()

optimizer = torch.optim.AdamW(params = model.parameters(), lr=CFG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-9, verbose=True)
age_model, age_best_score, age_best_loss = train(model, optimizer, train_loader, val_loader, scheduler, device, age_weights, model_name)

os.makedirs(f'/opt/ml/models/{project_idx}/{model_name}', exist_ok=True)
torch.save(age_model.module.state_dict(), f'/opt/ml/models/{project_idx}/{model_name}/[{CFG["MODEL"]}]_[score{age_best_score:.4f}]_[loss{age_best_loss:.4f}].pt')

gc.collect() # python 자원 관리 
torch.cuda.empty_cache() # gpu 자원관리

### Inference

In [ ]:
test_dir = '/opt/ml/input/data/eval'

In [ ]:
df = pd.read_csv(test_dir + '/info.csv')

In [ ]:
image_paths = [os.path.join(test_dir, 'images', img_id) for img_id in df.ImageID]

test_dataset = CustomDataset(image_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=8)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.to(device)
            
            logit = model(imgs)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [ ]:
mask_model_weights = torch.load(glob(f'/opt/ml/models/{project_idx}/Mask/*')[0])
mask_model = Mask()
mask_model.load_state_dict(mask_model_weights)

gender_model_weights = torch.load(glob(f'/opt/ml/models/{project_idx}/Gender/*')[0])
gender_model = Gender()
gender_model.load_state_dict(gender_model_weights)

age_model_weights = torch.load(glob(f'/opt/ml/models/{project_idx}/Age/*')[0])
age_model = Age()
age_model.load_state_dict(age_model_weights)

<All keys matched successfully>

In [44]:
mask_preds = inference(mask_model, test_loader, device)

In [45]:
gender_preds = inference(gender_model, test_loader, device)

In [46]:
age_preds = inference(age_model, test_loader, device)

In [47]:
"""
Mask               Gender          Age
- mask      : 0    - female : 0    - 30 미만         : 0 
- normal    : 1    - male   : 1    - 30 이상 60 미만 : 1
- incorrect : 2                    - 60 이상         : 2
"""
label_dict = {(0, 1, 0): 0, (0, 1, 1): 1, (0, 1, 2): 2, (0, 0, 0): 3, (0, 0, 1): 4, 
              (0, 0, 2): 5, (1, 1, 0): 6, (1, 1, 1): 7, (1, 1, 2): 8, (1, 0, 0): 9, 
              (1, 0, 1): 10, (1, 0, 2): 11, (2, 1, 0): 12, (2, 1, 1): 13, (2, 1, 2): 14, 
              (2, 0, 0): 15, (2, 0, 1): 16, (2, 0, 2): 17}

In [48]:
preds = []
for mask_ego_pred, gender_pred, age_pred in zip(mask_preds, gender_preds, age_preds):
    temp = (mask_ego_pred, gender_pred, age_pred)
    preds.append(label_dict[temp])

In [49]:
df['ans'] = preds
df.to_csv(os.path.join(test_dir, 'submits', f'{CFG["MODEL"]}_{project_idx}.csv'), index=False)
print('test inference is done!')

test inference is done!
